### The purpose of this notebook is to clean the data

In [226]:
import json
import os
import sys
from copy import deepcopy
import re

In [227]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [228]:
from update_sorted_stuff import *
from prune_helper import *

In [229]:
!ls

all_funcs.py
CNT_gs_viaf_dblp_wc_mgp_wikiquote_influ_semantic_research.json
del_attributes_list.json
oct_create_prelim_wiki_data.ipynb
old_CNT_gs_viaf_dblp_wc_mgp_wikiquote_influ_semantic_research.json
prune_dataframe.ipynb
prune_helper.py
purify_functions.py
__pycache__
sample_and_freqs_for_attributes.json
store_sorted_freqs.ipynb
test_names.py
test_purify_functions.ipynb
update_sorted_stuff.py


In [230]:
FILE_PATH="./old_CNT_gs_viaf_dblp_wc_mgp_wikiquote_influ_semantic_research.json"
with open(FILE_PATH, 'r') as fd:
    df=json.load(fd)

 q_str=re.sub(r'\(Q.*\)', '', q_str)
    q_str=re.sub(r'\(P.*\)', '', q_str)

In [231]:
def save_file():
    global df
    del_useless()
    old_df=deepcopy(df)
    df=[]
    for curr_elem in old_df:
        now_elem=dict()
        for curr_key, curr_val in curr_elem.items():
            the_key=curr_key[:]
            the_key=re.sub(r'\(Q.*\)', '',the_key)
            the_key=re.sub(r'\(P.*\)', '',the_key)
            now_elem[the_key]=deepcopy(curr_val)
        df.append(deepcopy(now_elem))
    FILE_PATH="./CNT_gs_viaf_dblp_wc_mgp_wikiquote_influ_semantic_research.json"
    with open(FILE_PATH, 'w') as fd:
        json.dump(df, fd, indent=4)

In [232]:
def inspect_attr(attr_name):
    FILE_PATH="./CNT_gs_viaf_dblp_wc_mgp_wikiquote_influ_semantic_research.json"
    with open(FILE_PATH, 'r') as fd:
        tmp_df=json.load(fd)
    for curr_p in tmp_df:
         if attr_name in curr_p:
                print(curr_p[attr_name])

### First delete all decided redundant attributes

In [233]:
def del_useless():
    with open("del_attributes_list.json",'r') as del_fd:
        del_arr=json.load(del_fd)

    for curr_elem in df:
        # go through each row and delete redundant columns
        for unlucky_key in del_arr:
            if unlucky_key in curr_elem:
                del curr_elem[unlucky_key]   

In [234]:
del_useless()

### Remove URL and cross-ref from dblp data

In [235]:
for curr_elem in df:
    # go through each row and delete redundant columns
    key_name="dblp_publications"
    if key_name in curr_elem:
        curr_elem[key_name]=remove_url_and_crossref_dblp(curr_elem[key_name])

### Handle dob conflicts and in case of conflict, mark it as UNRELIABLE and give priority to Wikipedia

In [236]:
for curr_elem in df:
    # go through each row and delete redundant columns
    key_name="person_dob"
    if key_name in curr_elem:
        curr_elem[key_name]=handle_person_dob(curr_elem[key_name])

for curr_elem in df:
    # go through each row and delete redundant columns
    key_name="person_date_of_death"
    if key_name in curr_elem:
        curr_elem[key_name]=handle_person_dob(curr_elem[key_name])   

### Merge the awards list from wikidata and the one obtained from research.com

In [237]:
df=[make_awards_consistent(x) for x in df]

##### Make associated webpage consistent using data from WIkidata and also the homepage mentioned in Google Scholar

In [238]:
df=[make_websites_consistent(x) for x in df]

### Remove student Ids, descendants in mgp_data

In [239]:
for curr_elem in df:
    # go through each row and delete redundant columns
    key_name="mgp_data"
    if key_name in curr_elem:
        curr_elem[key_name]['data']=clean_mgp_data(curr_elem[key_name]['data'])

### delete attribute of image does not exist

In [240]:
for curr_elem in df:
    # go through each row and delete redundant columns
    key_name="person_image_links"
    if key_name in curr_elem:
        if curr_elem[key_name]['data']==[] or curr_elem[key_name]['data']==None:
            del curr_elem[key_name]

### handle mgp_data

In [241]:
df=[expose_mgp_data(x) for x in df]

### Employer orgs, due to redundancy, face repetition
* {'data': ['Allen Institute for Artificial Intelligence', 'University of Washington', 'Allen Institute for AI, United States'], 'num_sources': 2}
* {'data': ['University of Texas at Austin', 'University of Texas at Austin, United States'], 'num_sources': 2}

### Also, need to merge employer_orgs with ORGs mentioned in dblp page, google scholar page etc

In [242]:
for curr_elem in df:
    # go through each row and delete redundant columns
    curr_elem=proper_setup_employer_orgs(curr_elem)
    key_name="employer_orgs"
    if key_name in curr_elem:
        curr_elem[key_name]['data']=prune_employer_orgs(curr_elem[key_name]['data'])

### merge advisor names taking care of abbreviations etc

In [243]:
df=[map_advisor(x) for x in df]
df=[map_students(x) for x in df]
df=[map_spouse(x) for x in df]
df=[merge_cats(["person_birth_place","place_of_birth"],x) for x in df]

In [244]:
#inspect_attr("person_birth_place")

In [245]:
save_file()
sort_stuff()

In [246]:
df[0]

{'wikipedia_page_url': {'data': 'https://en.wikipedia.org/wiki/Abraham_Silberschatz',
  'num_sources': 1},
 'wiki_pages_accessible_from_person_wiki_page': {'data': ['C. Mohan',
   'University of texas at austin',
   'VIAF (identifier)',
   'Database System Concepts',
   'SUDOC (identifier)',
   'Stony Brook University',
   'State University of New York',
   'Haifa, Israel',
   'Raghu Ramakrishnan',
   'Computer Science',
   'Yale University',
   'Database systems',
   'ACM Fellow',
   'RERO (identifier)',
   'Research',
   'ISNI (identifier)',
   'Bell Labs',
   'Doctoral advisor',
   'Computer scientist',
   'Operating systems',
   'Google Scholar'],
  'num_sources': 1},
 'person_name': {'data': ['Avi Silberschatz',
   'Abraham Silberschatz',
   'Abraham_Silberschatz',
   'Abraham'],
  'num_sources': 1},
 'backlinks_to_other_wiki_pages': {'data': ['Program counter',
   'Talk:Abraham Silberschatz',
   'List of fellows of the Association for Computing Machinery',
   'List of Stony Brook